In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras


In [3]:
# Load the mnist dataset.
train_ds, test_ds = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
)


d:\motaz\anaconda3\envs\mlflow\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  75%|███████▌  | 3/4 [00:01<00:00,  2.86 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:04<00:00,  1.01s/ url]


Dataset mnist downloaded and prepared to C:\Users\Motaz PC\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
def preprocess_fn(data):
    image = tf.cast(data["image"], tf.float32) / 255
    label = data["label"]
    return (image, label)


train_ds = train_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess_fn).batch(128).prefetch(tf.data.AUTOTUNE)


# model definition

In [5]:
input_shape = (28, 28, 1)
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)


In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(0.001),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Set up tracking

In [7]:
import mlflow
mlflow.set_tracking_uri(uri="http://localhost:8080")

#  Method 1: MLflow Auto Logging


In [8]:
# Choose any name that you like.
mlflow.set_experiment("/mlflow-tf-keras-mnist")

mlflow.tensorflow.autolog()

model.fit(x=train_ds, epochs=3)


2024/07/26 19:02:18 INFO mlflow.tracking.fluent: Experiment with name '/mlflow-tf-keras-mnist' does not exist. Creating a new experiment.
2024/07/26 19:02:18 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.
2024/07/26 19:02:18 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following val

Epoch 1/3
468/469 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.7371 - sparse_categorical_accuracy: 0.7653

2024/07/26 19:02:26 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.7355 - sparse_categorical_accuracy: 0.7658
Epoch 2/3
466/469 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.1135 - sparse_categorical_accuracy: 0.9659

2024/07/26 19:02:31 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.1134 - sparse_categorical_accuracy: 0.9659
Epoch 3/3
466/469 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0838 - sparse_categorical_accuracy: 0.9747

2024/07/26 19:02:37 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - loss: 0.0838 - sparse_categorical_accuracy: 0.9747


2024/07/26 19:02:37 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: tuple index out of range
2024/07/26 19:02:37 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


# score 

In [9]:
score = model.evaluate(test_ds)

print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]: .2f}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0464 - sparse_categorical_accuracy: 0.9839
Test loss: 0.0458
Test accuracy:  0.98


# Method 2: Log with MLflow Callback


In [10]:
from mlflow.tensorflow import MlflowCallback

# Turn off autologging.
mlflow.tensorflow.autolog(disable=True)

with mlflow.start_run() as run:
    model.fit(
        x=train_ds,
        epochs=2,
        callbacks=[MlflowCallback(run)],
    )


Epoch 1/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0699 - sparse_categorical_accuracy: 0.9784
Epoch 2/2
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0619 - sparse_categorical_accuracy: 0.9817


# Customize the MLflow Callback

In [11]:
import math


# Create our own callback by subclassing `MlflowCallback`.
class MlflowCustomCallback(MlflowCallback):
    def on_epoch_end(self, epoch, logs=None):
        if not self.log_every_epoch:
            return
        loss = logs["loss"]
        logs["log_loss"] = math.log(loss)
        del logs["loss"]
        self.metrics_logger.record_metrics(logs, epoch)

In [12]:
with mlflow.start_run() as run:
    run_id = run.info.run_id
    model.fit(
        x=train_ds,
        epochs=2,
        callbacks=[MlflowCustomCallback(run)],
    )

Epoch 1/2
464/469 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0567 - sparse_categorical_accuracy: 0.9826

2024/07/26 19:10:21 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'metrics_logger'


469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0567 - sparse_categorical_accuracy: 0.9826 - log_loss: -2.9134
Epoch 2/2
466/469 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0510 - sparse_categorical_accuracy: 0.9851

2024/07/26 19:10:25 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during autologging: 'MlflowCustomCallback' object has no attribute 'metrics_logger'


469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0510 - sparse_categorical_accuracy: 0.9851 - log_loss: -3.0009
